### training and test data stats

In [1]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt

# DESIGNEDDATAPATH = '../designed_data/cad/'

In [3]:
constructs = {}
construct_names = [
    'sexism',
    'hatespeech'
    ]
perplexity = 10 # 1, 5, 10, 15, 20
runs = 5

In [4]:
DESIGNEDDATAPATH = '../data/data/%s/train/paired_cads_flant5.csv' %construct_names[0]

In [7]:
# ! pip install pickle5

In [9]:
paired_data = {}

for construct in construct_names:
    train_data_path = '../../autoCAD/data/data/data/%s/train/paired_cads_mixed.csv' %construct
    paired_data[construct] = pd.read_csv(train_data_path, sep = '\t')
    paired_data[construct]['polyjuice_id'] = [str(i)+'p' for i in paired_data[construct]['original_id']]
    paired_data[construct]['chatgpt_id'] = [str(i)+'gpt' for i in paired_data[construct]['original_id']]

In [10]:
paired_data[construct].columns

Index(['Unnamed: 0', 'original_id', 'counterfactual_id', 'original_text',
       'counterfactual_text', 'original_label', 'counterfactual_label', 'diff',
       'negation_additions', 'negation_deletions', 'affect word_additions',
       'affect word_deletions', 'gender word_additions',
       'gender word_deletions', 'identity word_additions',
       'identity word_deletions', 'hedges_additions', 'hedges_deletions',
       'hate words_additions', 'hate words_deletions', 'polyjuice',
       'polyjuice_label', 'chatgpt', 'chatgpt_label', 'flant5', 'flant5_label',
       'polyjuice_text', 'chatgpt_text', 'flant5_text', 'mixed_cad_text',
       'mixed_cad_type', 'mixed_cad_label', 'polyjuice_id', 'chatgpt_id'],
      dtype='object')

In [11]:
for construct in construct_names:
    print(construct)
    print(paired_data[construct].groupby('original_label').size())
    print()
    print(paired_data[construct].groupby('counterfactual_label').size())
    print()
    print(paired_data[construct].groupby('polyjuice_label').size())
    print()
    print(paired_data[construct].groupby('chatgpt_label').size())
    print()
    print(paired_data[construct].groupby('flant5_label').size())
    print('---------------------------------------------------------')

sexism
original_label
non-sexist    1610
sexist        1244
dtype: int64

counterfactual_label
MAR            596
NMAR          1610
non-sexist     648
dtype: int64

polyjuice_label
MAR            553
NMAR          1610
non-sexist     691
dtype: int64

chatgpt_label
MAR              2
NMAR          1610
non-sexist    1242
dtype: int64

flant5_label
MAR            103
NMAR          1610
non-sexist    1141
dtype: int64
---------------------------------------------------------
hatespeech
original_label
hate        6524
not hate    5767
dtype: int64

counterfactual_label
hate        5780
not hate    6511
dtype: int64

polyjuice_label
MAR         5613
hate        3282
not hate    3396
dtype: int64

chatgpt_label
MAR          175
hate        5686
not hate    6430
dtype: int64

flant5_label
MAR         1069
hate        5136
not hate    6086
dtype: int64
---------------------------------------------------------


In [2]:
test_set_data = {}
constructs = ['sexism', 'hatespeech']

test_set_data['sexism'] = pd.read_csv('../../autoCAD/results/intermediate/sexism_results (3).csv', sep = '\t')
test_set_data['hatespeech'] = pd.read_csv('../../autoCAD/results/intermediate/hatespeech_results (3).csv',  sep = '\t')

In [14]:
for construct in constructs:
    print(test_set_data[construct].groupby(['dataset', construct]).size())
    print()

dataset          sexism    
hatecheck        non-sexist      136
                 sexist          373
in_domain        non-sexist      690
                 sexist          534
out_of_domain    non-sexist     1800
                 sexist         1636
out_of_domain_2  non-sexist     5856
                 sexist          699
out_of_domain_3  non-sexist    15146
                 sexist         4854
dtype: int64

dataset          hatespeech
hatecheck        hate           2563
                 not hate       1165
in_domain        hate            471
                 not hate        464
out_of_domain    hate          14614
                 not hate      19162
out_of_domain_2  hate           1260
                 not hate       1740
out_of_domain_3  hate           5256
                 not hate      17067
out_of_domain_4  hate           1267
                 not hate       5738
dtype: int64



In [15]:
test_set_data[construct]['dataset'].unique()

array(['in_domain', 'out_of_domain', 'out_of_domain_2', 'out_of_domain_3',
       'out_of_domain_4', 'hatecheck'], dtype=object)

In [16]:
test_set_data['sexism'].groupby(['sexism', 'dataset']).size().unstack()

dataset,hatecheck,in_domain,out_of_domain,out_of_domain_2,out_of_domain_3
sexism,,,,,
non-sexist,136,690,1800,5856,15146
sexist,373,534,1636,699,4854


In [17]:
test_set_data['hatespeech'].groupby(['hatespeech', 'dataset']).size().unstack()

dataset,hatecheck,in_domain,out_of_domain,out_of_domain_2,out_of_domain_3,out_of_domain_4
hatespeech,,,,,,
hate,2563,471,14614,1260,5256,1267
not hate,1165,464,19162,1740,17067,5738


### Delete original and counterfactuals in the datasets to respect licenscing agreements

In [17]:
test_set_data = {}
constructs = ['sexism', 'hatespeech']

test_set_data['sexism'] = pd.read_csv('../results/intermediate/sexism_results.csv', sep = '\t')
test_set_data['hatespeech'] = pd.read_csv('../results/intermediate/hatespeech_results.csv',  sep = '\t')

In [18]:
for construct in constructs:
    test_set_data[construct] = test_set_data[construct].drop('text', axis=1)
    test_set_data['sexism'].to_csv('../results/intermediate/sexism_results.csv', sep = '\t')
    test_set_data['hatespeech'].to_csv('../results/intermediate/hatespeech_results.csv',  sep = '\t')

In [20]:
list(test_set_data[construct].columns)

['_id',
 'chatgpt_labels',
 'hatespeech',
 'OG_Linear SVM_0_predictions',
 'OG_Linear SVM_0_probabilities',
 'OG_Linear SVM_1_predictions',
 'OG_Linear SVM_1_probabilities',
 'OG_Linear SVM_2_predictions',
 'OG_Linear SVM_2_probabilities',
 'OG_Linear SVM_3_predictions',
 'OG_Linear SVM_3_probabilities',
 'OG_Linear SVM_4_predictions',
 'OG_Linear SVM_4_probabilities',
 'OG_transformer_0_predictions',
 'OG_transformer_0_probabilities',
 'OG_transformer_1_predictions',
 'OG_transformer_1_probabilities',
 'OG_transformer_2_predictions',
 'OG_transformer_2_probabilities',
 'OG_transformer_3_predictions',
 'OG_transformer_3_probabilities',
 'OG_transformer_4_predictions',
 'OG_transformer_4_probabilities',
 'CAD_Linear SVM_0_predictions',
 'CAD_Linear SVM_0_probabilities',
 'CAD_Linear SVM_1_predictions',
 'CAD_Linear SVM_1_probabilities',
 'CAD_Linear SVM_2_predictions',
 'CAD_Linear SVM_2_probabilities',
 'CAD_Linear SVM_3_predictions',
 'CAD_Linear SVM_3_probabilities',
 'CAD_Linear SVM